In [104]:
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.tools as plotly_tools
import plotly.graph_objs as go
from pythresh.thresholds.zscore import ZSCORE
import datapane as dp

from preprocessing.awr_preprocessing.load_profile_processing import LoadProcessor
from preprocessing.awr_preprocessing.sql_processing import SqlProcessor
from preprocessing.awr_preprocessing.wait_classes_processing import WaitClassProcessor
from preprocessing.awr_preprocessing.foreground_events_waits_processing import ForegroundEventWaitProcessor

In [70]:
def melt_df(df):
    df_melt = df.melt(value_vars=df.columns.tolist(), ignore_index=False)
    # df_melt['timestamp'] = df_melt.index
    df_melt = df_melt.reset_index()

    return df_melt

In [71]:
def generate_overview(df_melt):
    fig = px.line(df_melt, x='index', y='value', facet_col='variable',
        facet_col_wrap=5,
        color='variable',
        facet_col_spacing=0.04,  facet_row_spacing=0.04,
        # width=1400,
        height=600
    )

    # fig.for_each_annotation(lambda x: x.update(text=x.text.split("=")[-1]))
    fig.update_xaxes(matches='x', tickformat='')
    # fig.update_yaxes(showticklabels=True, matches=None)

    fig.update_layout(autosize=True, template='plotly')

    fig.update_yaxes(matches=None, showticklabels=True, visible=True)
    # fig.update_annotations(font=dict(size=16))
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    # hide subplot y-axis titles and x-axis titles
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.YAxis:
            fig.layout[axis].title.text = ''
        if type(fig.layout[axis]) == go.layout.XAxis:
            fig.layout[axis].title.text = ''

    # fig.write_html(Path('res/awr/facet.html'))
    # fig.show()
    return fig

In [72]:
def generate_big_plots(df_melt):
    fig = px.line(df_melt, x='index', y='value', facet_col='variable',
        facet_col_wrap=2,
        color='variable',
        facet_col_spacing=0.05,
        facet_row_spacing=0.02,
        # width=1400,
        height=5000
    )

    # fig.for_each_annotation(lambda x: x.update(text=x.text.split("=")[-1]))
    fig.update_xaxes(matches='x', tickformat='')
    # fig.update_yaxes(showticklabels=True, matches=None)

    fig.update_layout(
        showlegend=False,
        # autosize=True
        template='plotly'
    )

    fig.update_yaxes(matches=None, showticklabels=True, visible=True)
    # fig.update_annotations(font=dict(size=16))
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.for_each_xaxis(lambda xaxis: xaxis.update(showticklabels=True))
    # hide subplot y-axis titles and x-axis titles
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.YAxis:
            fig.layout[axis].title.text = ''
        # if type(fig.layout[axis]) == go.layout.XAxis:
        #     fig.layout[axis].title.text = ''

    # fig.write_html(Path('res/awr/plotlist.html'))
    # fig.show()
    return fig

In [91]:
def generate_wait_classes_plot(df):
    df_melt = melt_df(df)

    fig = px.line(df_melt, x='index', y='value', color='variable',
        # width=1400,
        height=600
    )

    fig.update_layout(
        autosize=True, template='plotly',
        xaxis_title="",
        yaxis_title="Total Wait Time (sec)"
    )

    # fig.write_html(Path('res/awr/facet.html'))
    # fig.show()
    return fig

In [74]:
def generate_peaks_figures(df, metric_list):
    df = df[metric_list]
    figure_list = []
    peak_list = []
    for metric in metric_list:
        thres = ZSCORE()
        labels = thres.eval(df[metric])
        peak_timestamps = df[labels == 1][[metric]]


        fig = px.line(df[[metric]])
        fig = px.scatter(peak_timestamps,
                            color_discrete_sequence=['#EF553B'],
                            labels={
                                "value": metric,
                                "index": "time"
                            },
                            title=metric
                          ).add_trace(fig.data[0])

        fig.update_layout(
            showlegend=False,
            autosize=True,
            template='plotly'
        )

        figure_list.append(fig)
        peak_list.append(peak_timestamps)

    return peak_list, figure_list

In [75]:
def generate_wait_classes_pie(df):
    fig = px.pie(df, values='% DB time', names='Wait Class')
    fig.update_layout(autosize=True, template='plotly')
    # fig.show()
    return fig

In [76]:
l = LoadProcessor('load-prc')
df = l.grouped_df

In [77]:
df_melt = melt_df(df)
fig1 = generate_overview(df_melt)
fig2 = generate_big_plots(df_melt)

metric_list = ['DB Time(s)', 'Hard parses (SQL)', 'Read IO (MB)', 'Write IO (MB)']
peak_list, figure_list = generate_peaks_figures(df, metric_list)

In [78]:
# summary_table_1 = df.describe()
# summary_table_1 = summary_table_1\
#     .to_html()\
#     .replace('<table border="1" class="dataframe">','<table class="table table-striped">') # use bootstrap styling

In [79]:
# reportpath = Path('res/awr/report.html')
# if os.path.exists(reportpath):
#     os.remove(reportpath)
#
# with open(reportpath, 'a') as f:
#     f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn', div_id='facets'))
#     f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn', div_id='big-plots'))

In [80]:
### SQL Processor
p = SqlProcessor('sql-proc')
t1,t2,t3 = p.get_top_n(n=3, start=str(peak_list[0].index[0]), end=str(peak_list[0].index[0]))

In [81]:
# reportpath = Path('res/awr/report.html')
# if os.path.exists(reportpath):
#     os.remove(reportpath)
#
# with open(reportpath, 'a') as f:
#     f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn', div_id='facets'))
#     f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn', div_id='big-plots'))
#     f.write(df.to_html())

In [92]:
wc = WaitClassProcessor('waitClasses-prc')
fig3 = generate_wait_classes_plot(wc.grouped_df)

In [107]:
fw = ForegroundEventWaitProcessor('feWait-prc')

In [110]:
def generate_timestamp_block_tables(t1, t2, t3, pie_df, fw_df, label):
    block = dp.Group(
        dp.Group(
            dp.Blocks(
                dp.Text("""## Wait Classes % DB time"""),
                dp.Plot(generate_wait_classes_pie(pie_df))
            ),
            dp.Blocks(
                dp.Text("""## Top 10 Foreground Events by Total Wait Time"""),
                dp.DataTable(fw_df)
            ),
            widths=[40, 60],
            columns=2
        ),
        dp.Text("""## Top CPU Time (s)"""),
        dp.DataTable(t1),
        dp.Text("""## Top Elapsed Time (s)"""),
        dp.DataTable(t2),
        dp.Text("""## Top User I/O Time (s)"""),
        dp.DataTable(t3),
        label=label,
        columns=1
    )

    return block


def generate_timestamps_block_list(sqlProcessor, waitClassesProcessor, foregroundEventWaitProcessor,
                                   peak_list, n=3):
    block_list = []
    for ts in peak_list:
        ts = str(ts)
        t1, t2, t3 = sqlProcessor.get_top_n(n=n, start=ts, end=ts)
        pie_df = waitClassesProcessor.get_db_time_percentages(ts)
        fw_df = foregroundEventWaitProcessor.filter_by_date(foregroundEventWaitProcessor.df,
                                                            start=ts, end=ts)
        block = generate_timestamp_block_tables(t1, t2, t3, pie_df, fw_df, label=ts)
        block_list.append(block)

    return block_list


def generate_timestamps_dropdown(block_list):
    dropdown = dp.Select(
        blocks=block_list,
        type=dp.SelectType.DROPDOWN
    )

    return dropdown


def generate_critical_timestamp_block(sqlProcessor, waitClassesProcessor, foregroundEventWaitProcessor,
                                      figure, peak_df, label):
    peak_list = peak_df.index
    block_list = generate_timestamps_block_list(sqlProcessor, waitClassesProcessor, foregroundEventWaitProcessor,
                                                peak_list, n=10)
    dropdown = generate_timestamps_dropdown(block_list)

    block = dp.Group(
        dp.Group(
            dp.Plot(figure),
            dp.Table(peak_df),
            widths=[75, 25],
            columns=2
        ),
        dropdown,
        label=label,
        columns=1
    )

    return block

In [111]:
b1 = generate_critical_timestamp_block(p, wc, fw, figure_list[0], peak_list[0], metric_list[0])
b2 = generate_critical_timestamp_block(p, wc, fw, figure_list[1], peak_list[1], metric_list[1])
b3 = generate_critical_timestamp_block(p, wc, fw, figure_list[2], peak_list[2], metric_list[2])
b4 = generate_critical_timestamp_block(p, wc, fw, figure_list[3], peak_list[3], metric_list[3])

In [112]:
#TODO add SYSTEM INFO
#TODO add TABLESPACE IO
#TODO reformat all "Critical Timestamps" tables
#TODO add log io stats

datapane_app = dp.Blocks(
    dp.Page(
        title="Overview",
        blocks=["### Load Profile", dp.Plot(fig1, scale=1.5)]
    ),
    dp.Page(
        dp.Plot(fig2, scale=1.5),
        title="Load Profile"
    ),
    dp.Page(
        dp.Select(
            blocks=[b1,b2,b3,b4]
        ),
        title="Critical Timestamps"
    ),
    dp.Page(
        dp.Plot(fig3, scale=1.5),
        title="Wait Classes",
        blocks=["### Total Wait Time for each Wait Class", dp.Plot(fig3, scale=1.5)]
    )
)



dp.save_report(
    datapane_app,
    path="res/awr/awr_report.html",
    formatting=dp.Formatting(
        light_prose=False,
        accent_color="orange",
        bg_color="#EEE",
        # text_alignment=dp.TextAlignment.RIGHT,
        font=dp.FontChoice.MONOSPACE,
        width=dp.Width.FULL,
    )
)

C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\datapane\common\df_processor.py:28: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\datapane\common\df_processor.py:28: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\datapane\common\df_processor.py:28: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\datapane\common\df_processor.py:28: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Us

App saved to ./res/awr/awr_report.html

In [86]:
# import datapane as dp
#
# datapane_app = dp.App(
#     dp.Page(
#         title="Overview",
#         blocks=["### Load Profile", dp.Plot(fig1, scale=1.5)]
#     ),
#     dp.Page(
#         dp.Plot(fig2, scale=1.5),
#         title="All Metrics"
#     ),
#     dp.Page(
#         dp.Select(
#             blocks=[
#                 dp.Blocks(
#                     dp.Group(
#                         dp.Plot(figure_list[0]),
#                         dp.Table(peak_list[0]),
#                         widths=[75, 25],
#                         columns=2
#                     ),
#                     dp.Group(
#                         dp.Plot(figure_list[1]),
#                         dp.Table(peak_list[1]),
#                         widths=[75, 25],
#                         columns=2
#                     ),
#                     dp.Group(
#                         dp.Plot(figure_list[2]),
#                         dp.Table(peak_list[2]),
#                         widths=[75, 25],
#                         columns=2
#                     ),
#                     dp.Group(
#                         dp.Plot(figure_list[3]),
#                         dp.Table(peak_list[3]),
#                         widths=[75, 25],
#                         columns=2
#                     )
#                 ),
#                 dp.Blocks(
#                     dp.Text('Prova'),
#                     dp.Table(t1),
#                     dp.Table(t2),
#                     dp.Text('Prova'),
#                     dp.Table(t3),
#                 )
#             ]
#         ),
#         title="Critical Timestamps"
#     ),
#
# )
# dp.save_report(
#     datapane_app,
#     path="styling-python-api.html",
#     formatting=dp.Formatting(
#         light_prose=False,
#         accent_color="orange",
#         bg_color="#EEE",
#         text_alignment=dp.TextAlignment.RIGHT,
#         font=dp.FontChoice.MONOSPACE,
#         width=dp.Width.FULL,
#     )
# )